이미지 분류기를 직접 작성할 수도 있으나 이미 전문적으로 작성된 모델을 불러, 일부 영역의 학습을 통해 작성자가 원하는 모델을 만들수 있다.

전이학습을 하면 방대한 데이터, 우수한 성능의 시스템이 없어도 우수모델 사용이 가능

방법 첫번째 : 특성 추출 기법 : 사전 훈련된 모델의 마지막 완전 연결층 부분만 새로 학습

방법 두번째 : 미세조정기법
베이스 모델은 MobileNet v2

In [ ]:
%pip install tensorflow-datasets

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

(raw_train, raw_validation, raw_test), metadata = tfds.load('cats_vs_dogs', split= ['train[:80%]','train[80%:90%]','train[:90%]'],with_info=True, as_supervised=True)


In [ ]:
print(raw_train)
print(raw_train.take(1))
print(raw_validation)
print(raw_test)
print(metadata)

mobile v2가 개 고양이 분류하는 곳

In [ ]:
get_label_name= metadata.features['label'].int2str

for image, label in raw_train.take(1):
    plt.figure()
    plt.imshow(image)
    print(label.numpy())
    print(get_label_name(label)) # dog은 1 cat은 0
    plt.title(label.numpy())
    plt.show()


현재 데이터는 용량이 매우 크기 때문에 ram에 모두 로딩이 불가. 데이터 파이프라인을 이용하면 이미지를 일부만 적당히 연속해서 읽어 처리
읽어서 처리하고 ram에서 내보내기. - 일련의 처리 과정을 할 수 있다. -> 이미지 포매팅을 해야함




In [ ]:
import tensorflow as tf
#이미지 포맷팅, 셔플링 처리 : MobileNet v2가 원하기 때문

IMG_SIZE = 160


def format_exam(image, laebl):
    image = tf.cast(image, tf.float32)
    image = (image / 127.5)-1 #이미지 ㅔ이터를 실수화 하고 -1~1사이로 크기를 맞춤
    image=tf.image.resize(image, (IMG_SIZE,IMG_SIZE))
    return image, label

train = raw_train.map(format_exam)
validation = raw_validation.map(format_exam)
test=raw_test.map(format_exam)

#이미지 배치
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE=1000

train_batches = train.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

for image_batch, label_batch in train_batches.take(1):
    pass

print(image_batch.shape,'',label_batch.shape)


In [ ]:
# base model 작성 : MobileNet v2 - 대량ㅇ의 자료로 충분히 학습된 분류 모델
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model= tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
# 입력층 -> CNN층(특징 추출) -> 완전연결층
# include_top = False를 하면 입력층 -> CNN층(특징 추출)  => 학습이 끝난 녀석을 가져온 것이기에 그냥 써먹어야 한다.

feature_batch=base_model(image_batch) # (32,160,160,3) 형태의 이미지 특징 반환
print(feature_batch) # shape= (32,5,5,1280)


In [ ]:
# 계층 동결
base_model.trainable = False # MobileNetV2 는 학습이 끝났으므로 우리가 만들 모델에서는 학습 안함
# print(base_model.summary())

# 모델링
# 2
global_average_layer = tf.keras.layers.GlobalAveragePooling2D() # AveragePooling2D보다 급격하게 feature의 수를 줄임 - 우리가 생각하는 것보다 훨씬 빠름
feature_batch_average=global_average_layer(feature_batch)
print(feature_batch_average.shape)

#3 순전파를 하고 난 다음에 형태는 최종적으로 빠져나갈때 1로 나감
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape) #(32, 1)

#3개의 레이어를 넣으면 모델이 만들어짐
model = tf.keras.Sequential([
    base_model, #특징 추출 베이스 모델
    global_average_layer, # 풀링 레이어
    prediction_layer # 완전연결층
])



In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

print(model.summary())

#Trainable params: 1281 (5.00 KB) 순전파에는 영향을 준다

#현재 모델 성능
validation_step= 20
loss0, accuracy0 = model.evaluate(validation_batches, steps=validation_step)
print("loss0 :", loss0)
print("accuracy0 :", accuracy0)

initial_epochs=10
history= model.fit(train_batches, epochs= initial_epochs, validation_data= validation_batches)



In [ ]:
# 학습시 제공된 정보로 시각화
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(5,5))
plt.subplot(2, 1, 1)
plt.plot(acc, label='acc')
plt.plot(val_acc, label='val_acc')
plt.legend(loc='lower right')

plt.subplot(2,1,2)
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.legend(loc='upper right')
plt.show()

# 파인튜닝이란?
 사전 학습 모델(pre-trained model)에 도메인 특화 데이터를 추가 학습시켜 맞춤형 모델로 업데이트 하는 것을 의미
 전이 학습이 끝난, 모델에 대해 레이어 일부를 재조정
 즉, 기존에 있는 모델의 가장 끝단에 있는 레이어의 일부도 학습에 참여시키는 것
 베이스 모델의 끝단 레이어 일부도 학습에 처리


In [ ]:
base_model.trainable=True # base모델의 학습 동결을 해제

print('베이스 모델 레이어 수 : ', len(base_model.layers)) #154
fine_tune_at =100

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable= False #베이스 모델의 앞단 100개는 학습동결시킴

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # from_logits를 False로 설정
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate / 10),
    metrics=['accuracy']
)

# 모델 요약 출력
print(model.summary())


In [ ]:
# 파인 튜닝 학습
fine_tune_epochs = 5
total_epochs = initial_epochs + fine_tune_epochs

history_fine= model.fit(train_batches, epochs=total_epochs,initial_epoch= history.epoch[-1],validation_data= validation_batches)

#시각화
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']
loss+= history_fine.history['loss']
val_loss+= history_fine.history['val_loss']

plt.figure(figsize=(5,5))
plt.subplot(2, 1, 1)
plt.plot(acc, label='acc')
plt.plot(val_acc, label='val_acc')
plt.ylim([0.8,1])
plt.plot([initial_epochs-1,initial_epochs-1],plt.ylim(),label='start fine tuning')
plt.legend(loc='lower right')

plt.subplot(2,1,2)
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.legend(loc='upper right')
plt.xlabel('epochs')
plt.show()